# 卡尔曼滤波介绍

现代系统大多数都配备了多个传感器，这些传感器根据一系列**测量值**提供**隐藏（未知）变量的估计**。例如，GPS接收机提供位置和速度估计，其中位置和速度是隐藏变量，卫星信号到达差值时间是测量值。

跟踪控制系统最大的挑战之一是**在不确定性存在的情况下对隐藏变量进行精确的估计**。在GPS接收机中，测量的不确定性取决于许多外部因素，如热噪声、大气效应、卫星位置的微小变化、接收机时钟精度等等。

卡尔曼滤波器是一种重要且常用估计算法。卡尔曼滤波器**根据不准确和不确定的测量结果来估计隐藏变量**。此外，基于过去的估计，卡尔曼滤波器还能提供对未来系统状态的预测。

滤波器以Rudolf E.Kalman（1930年5月19日-2016年7月2日）命名。1960年，卡尔曼发表了他著名的论文，描述了离散数据线性滤波问题的递归解。

如今，卡尔曼滤波器被用于跟踪目标（雷达）、定位和导航系统、控制系统、计算机图形学等等。

## 预测需要

在深入分析卡尔曼滤波器之前，我们先了解一下预测算法的必要性。

看一个例子，让我们假设一个雷达跟踪算法。

![](images/v2-58b622df2d0cd79149dc54fca1a66d3c_720w.jpg)

如图，雷达向目标发送铅笔形跟踪波束。假设航迹周期为5秒，因此每隔5秒，雷达会向目标的方向发送一个专用的跟踪波束，从而重新访问目标。

发射光束后，雷达估计目标的当前位置和速度。此外，雷达估计目标（或预测）在下一个跟踪光束的位置。

使用牛顿运动方程可以很容易地计算出目标的未来位置：
$$x=x_0+v_0\triangle t+\frac 1 2 a\triangle t^2$$
其中，x是目标位置，$x_0$是目标的初始位置, $v_0$是目标的初始速度, a是目标加速度, $\triangle t$ 是时间间隔。

三维空间中，运动方程可以写为：

\begin{equation}
\left\{
             \begin{array}{lr}
             x=x_0+v_{x0}\triangle t+\frac 1 2 a_x\triangle t^2 & \\
             y=y_0+v_{y0}\triangle t+\frac 1 2 a_y\triangle t^2 \\
             x=z_0+v_{z0}\triangle t+\frac 1 2 a_z\triangle t^2 &  
             \end{array}
\right.
\end{equation}

目标参数 [x,y,z,$v_x$,$v_y$,$v_z$,$a_x$,$a_y$,$a_z$] 被称为**系统状态 System State**。**当前状态**是预测**算法的输入**，**下一个状态**（下一时间间隔的目标参数）是**算法的输出**。上述方程组就是**动态模型 Dynamic Model**或者**状态空间模型State Space Model**动态模型描述了输入和输出之间的关系。

如果知道当前状态和动态模型，就可以很容易地预测目标下一个状态。但事情并没有那么简单。首先，雷达测量不是绝对准确的。它包括随机误差（或不确定性）。误差大小取决于许多参数，如雷达校准、波束宽度、回波大小等。测量中包含的误差称为**测量噪声Measurement Noise**。

此外，由于风、空气湍流、飞行员机动等外部因素，目标运动与运动方程没有严格对齐（意思是目标运动并不是完全与方程描述相符）。动态模型误差（或不确定性）称为**过程噪声Process Noise**。

由于测量噪声Measurement Noise和过程噪声Process Noise的影响，估计的目标位置与实际目标位置相差甚远。在这种情况下，雷达会将跟踪光束发送到错误的方向，并错过目标。

为了提高雷达跟踪性能，需要一种**考虑过程不确定性和测量不确定性的预测算法**。

最广泛使用的预测算法是卡尔曼滤波器。

## 数学基础补充

几个基本术语，如方差（variance）、标准差（standard deviation）、估计值（estimate）、准确度（accuracy）、精度（precision）、平均值（mean）和期望值（expected value）。

### 平均值与期望值

虽然平均值(mean)与期望值（expected value）是密切相关的术语。但是，它们是不同的。

例如，假设有五种不同的硬币——两个5美分的硬币和三个10美分的硬币，我们可以通过平均硬币的价值来轻松计算硬币的平均值。

但是这个结果不能被定义为期望值，因为系统状态（硬币值）**没有被隐藏**（想要表达的是确定的，此处**没有任何不确定性**），我们已经使用了所有的population（所有5枚硬币）来计算平均值。

我们想知道某件事发生的概率，就作实验，然后记录时间的频率，大数定律告诉我们频率极限是概率，即实际上在实验的时候，平均值是在变化的，是随机的，当试验次数无穷多时，平均值会收敛到一个常数。如果考虑某概率分布，作多次实验，随机实验样本空间A是{1,2,3,...n}。作N次实验记录各个结果出现的次数，分别为 $N_1$，$N_2$，$N_3$，……，$N_n$，平均值：
$$mean=\sum _{x=1}^n \frac{x*N_x}{N}$$
大数定律：
$$lim _{N->\infty}\frac{N_x}{N}=P_A(x)$$
推导可得到：
$$lim _{N->\infty} mean=lim _{N->\infty} \frac{x*N_x}{N}=\sum _{x=1}{n}x*P_A(x)$$
这就是期望值。

现在假设同一个人的五个不同的体重测量值：79.8kg，80kg，80.1kg，79.8kg和80.2kg。由于秤的随机测量误差，称重测量值不同。 我们不知道**准确的重量值**是多少，因为它是一个**隐藏变量Hidden Variable**。 但是，我们可以**通过平均尺度测量来估计重量**。

估计的结果是体重的期望值。

- 平均数经常使用希腊字母：μ
- 期望值使用字母：E

### 方差与标准差

方差用来度量随机变量与其期望值（即随机变量的期望值）之间的离散程度。

标准差是方差的平方根。标准差： $\sigma$ ，方差： $\sigma ^2$

这就不多说了。例如想比较两个高中篮球队的身高。假设我们要计算所有高中篮球运动员的平均值和方差。这是一项非常艰巨的任务，我们需要收集所有高中运动员的数据。但是，我们可以通过选择一个大的数据集并对这个数据集进行计算来估计参与者的平均值和方差。（样本估计全局）

随机选取的100名选手的数据集足以进行准确的估计。然而，当我们估计方差时，方差计算公式略有不同。我们不用N因子归一化，而是用N - 1因子归一化，原因可以参考：[Why divide the sample variance by N-1?](https://www.visiondummy.com/2014/03/divide-variance-n-1/)

事实证明，许多自然现象服从正态分布。通常，测量误差是正态分布的，因此**卡尔曼滤波器设计基于测量误差是正态分布的假设**。

### 估计、准确度与精度

- 估计（Estimate）：**评估系统的隐藏状态**。飞机的真实位置对观察者来说是隐藏的。我们可以用雷达等传感器来估计飞机的位置。采用多传感器和先进的估计跟踪算法（如卡尔曼滤波），可以显著提高估计精度。每一个测量或计算参数都是一个估计值。
- 准确度（Accuracy）：表明**测量值与真实值的接近程度**。
- 精度（Precision）：描述**同一参数的许多度量值中有多少可变性**。准确度和精度是估算的基础。

下图说明了准确度和精度。

![](images/AccuracyAndPrecision.png)

高精度系统的测量方差较低（即不确定度/离散程度/变化程度较低），而低精度系统的测量方差较大（即不确定度/离散程度/变化程度较高）。方差是由随机测量误差产生的。

低精度系统被称为**偏差biased系统**，因为它们的测量具有内置的系统误差（偏差）。

通过平均或平滑测量可以显著降低方差的影响。例如，如果我们使用一个具有随机测量误差的温度计来测量温度，我们可以进行多次测量并对测量的值进行平均。由于误差是随机的，所以有些测量值会高于真实值，而另一些测量值会低于真实值。我们做的测量越多，估计就越接近。

另一方面，如果温度计有偏差，估计将包括一个恒定的系统误差。

本教程中的所有示例都假定系统是**无偏差**的。

![](images/statistical_view.png)

- The measurement is a **random variable**, described by the **Probability Density Function (PDF)**.
- The **measurements mean** is the **Expected Value** of the random variable.
- The offset between the measurements mean and the true value is the **measurements accuracy** also known as **bia**s or **systematic measurement error**.
- The dispersion of the distribution is the **measurement precision**, also known as the **measurement noise** or **random measurement error** or **measurement uncertainty**.

## α−β−γ滤波器

用两个例子来介绍。

### 案例1： 称金子重量

看第一个简单的例子。在这个例子中，我们将**估计静态系统的状态**。静态系统是一个在**合理时间内不会改变其状态的系统**。例如，静态系统可以是一个塔，状态是它的高度。

在这个案例中，我们要估计金条的重量。我们有一个无偏差量度，也就是说，这个量度没有系统误差，但是在测量中依然会包含随机噪声。（误差分为内部外差/系统误差，外部误差/非系统误差，这就想说明的是非内部/系统误差）

在这个例子中，系统是金条，系统的状态是金条的重量。因为我们假设重量在短时间内不会改变，因此系统的动态模型是恒定的。

为了估计系统的状态（即金条重量），我们可以进行多次测量并取其平均值。

![](images/ex1_MeasurementsVsTrueVal.png)

在时间n时， $\hat{x}_{N,N}$ 的估计值是所有先前测量的平均值：
$$\hat{x}_{N,N}=\frac{1}{N}(z_1+z_2+\cdots+z_{N-1}+z_N)=\frac{1}{N}\sum _{n=1}^{N}(z_n)$$
其中:

- x	is the true value of the weight
- $z_n$	is the measurement value of the weight at time n
- $\hat{x}_{n,n}$	is the estimate of x at time n (the estimate is made after taking the measurement $z_n$ )
- $\hat{x}_{n,n-1}$	is the previous estimate of x that was made at time n−1 (the estimate was made after taking the measurement $z_{n-1}$)
- $\hat{x}_{n+1,n}$	is the estimate of the future state (n+1 ) of x. The estimate is made at the time n, right after the measurement $z_n$. In other words, $x_{n+1,n}$ is a **predicted state**

上面说明稍微有点绕，把我关键，即n是现在，现在及之前有估计有测量，现在之后只有估计。下标的表示上，$\hat x$之后第一个下标表示面向的时间即状态的时间，后面一个是做出动作的时间，比如$\hat{x}_{n,n-1}$是n-1时段的时候做出的估计，但是是对n时段的估计。

本例中的动态模型是恒定的，因此 $\hat{x}_{n,n}=\hat{x}_{n+1,n}$

为了估计$\hat{x}_{N,N}$，需要记住所有的历史测量值，假设我们没有笔和纸来记录测量值，我们也不能依靠记忆来记忆所有的历史测量值。我们**只想使用之前的估计值并添加一个小的调整**（在现实应用程序中，我们希望节省计算机内存）。我们可以用一个小的数学技巧来做到这一点：

||Notes|
|-|-|
|$\hat{x}_{N,N}=\frac 1 N\sum_{n=1}^{N}(z_n)$|Average formula: sum of N measurements divided by N|
|$=\frac 1 N(\sum _{n=1}^{N-1}(z_n)+z_N)$|Sum of the N−1 measurements plus the last measurement divided by N|
|$=\frac 1 N\sum _{n=1}^{N-1}(z_n)+\frac 1 N z_N$|Expand|
|$=\frac 1 N \frac{N-1}{N-1}\sum _{n=1}^{N-1}(z_n)+\frac 1 N z_N$|Multiply and divide by term N−1|
|$=\frac{N-1}{N} \frac{1}{N-1}\sum _{n=1}^{N-1}(z_n)+\frac 1 N z_N$|Reorder The 'orange' term is the previous estimate|
|$=\frac{N-1}{N}\hat{x}_{N,N-1}+\frac 1 N z_N$|Rewriting the sum|
|$=\hat{x}_{N,N-1}-\frac 1 N \hat{x}_{N,N-1}+\frac 1 N z_N$|Distribute the term N−1N|
|$=\hat{x}_{N,N-1}+\frac 1 N(z_N-\hat{x}_{N,N-1})$|Reorder|

$\hat{x}_{N,N-1}$ 表示在时间N时对x状态的预测，而这个状态预测是基于在时间N-1时的测量。换句话说， ${x}_{N,N-1}$ 是上一次对目前的估计值。在本例子中就是前N-1次测量均值。

上述方程是五个卡尔曼滤波器方程之一。它被称为**状态更新方程State Update Equation**（敲黑板：后面会一直用到这个方程，请务必熟练于心）。其含义如下：

![](images/ex1_stateUpdate.png)

系数 $\frac 1 N$ 对于我们的示例是特定的。稍后我们将讨论这个系数的重要作用，不过现在先指出，在卡尔曼滤波器中，这个因素被称为**卡尔曼增益(Kalman Gain)**。用 $K_n$ 表示。下标n表示卡尔曼增益随每次迭代而变化。

$K_n$ 的发现是Rudolf Kalman的主要贡献之一。

同时，在我们深入探讨卡尔曼滤波器之前，我们将使用希腊字母 $\alpha$ 来表示这个系数而不是 $K_n$ 。

因此，状态更新方程如下：
$$\hat x _{n,n}=\hat x _{n,n-1}+\alpha _n(z_n-\hat x_{n,n-1})$$
$z_n-\hat x_{n,n-1}$是measurement residual，也被称为innovation。innovation中包含新信息。

在我们的案例中， 1/N 随着N增加而减小。这意味着在开始时，我们没有足够的重量值信息，因此我们将估计值建立在测量的基础上。随着我们继续测量，由于 1/N 的减小，在估计过程中的每个连续测量的变化起的作用越来越小。直到某个点，新测量的贡献将变得微不足道。

继续看这个例子。在进行第一次测量之前，我们可以通过目估（粗略估计）金条的重量。这被称为初步猜测Initial Guess，这将是我们的第一次估计。

正如我们稍后将看到的，卡尔曼滤波器需要初始猜测作为预设，这个猜测可能非常粗略。

下图描述了本例中使用的估计算法。

![](images/ex1_estimationAlgorithm.png)

现在我们准备开始测量和估计过程。

#### 第0次迭代

1）初始化：

我们对金条重量的初步估计是1000g。滤波器初始化操作仅需一次，因此在下一次迭代中不需要它。
$$\hat{x}_{0,0}=1000g$$

2）预测:

金条的重量不会改变。因此，系统的动态模型是静态的。我们的下一个状态估计（预测）等于初始化：
$$\hat{x}_{1,0}=\hat{x}_{0,0}=1000g$$

#### 第1次迭代

用天平测量重量得到：
$$z_1=1030g$$
计算卡尔曼增益，如前所述，本例中，卡尔曼增益为 $\alpha _n = \frac 1 n$：
$$\alpha _1 = 1/1 =1$$
使用状态更新方程计算当前估计值：
$$\hat x_{1,1}=\hat x_{1,0}+\alpha _1(z_1-\hat x_{1,0})=1000+1(1030-1000)=1030g$$
注意：在这个特定的例子中，最初的猜测可以是任何数字。由于 $\alpha _1=1$ ，初始猜测在第一次迭代中被消除。

系统的动态模型是静态的，所以金条的重量不应该改变。下一个状态估计（预测）等于当前状态估计：
$$\hat x_{2,1}=\hat x_{1,1}=1030g$$

#### 第2次迭代

单位时间延迟后，上一次迭代的预测值predicted estimate将成为当前迭代的之前估计previous estimate：
$$\hat x_{2,1}=1030g$$
对重量进行第二次测量得到：
$$z_{2}=989g$$
计算卡尔曼收益Kalman gain:
$$\alpha _1 = 1/2$$
计算当前估计值：
$$\hat x_{2,2}=\hat x_{2,1}+\alpha _1(z_2-\hat x_{1,0})=1030+1/2(989-1030)=1009.5g$$
估计下一个状态：
$$\hat x_{3,2}=\hat x_{2,2}=1009.5g$$

#### 第3至第10次迭代

同样的过程迭代多次，这里就不赘述了，每次测量卡尔曼增益都会减小。因此，每次连续测量的贡献都低于先前测量的贡献。我们的重量接近于1010克。

![](images/ex1_MeasVsTrueVsEst.png)

可以看到，我们的估计算法对测量有平滑的影响（意思是，估计算法会使得测量值变化减小），并且它收敛到真实值。

### 案例2 ：跟踪一维匀速飞行器

现在做一个随时间改变状态的动态系统。在这个例子中，我们将使用α-β filter在一维上跟踪等速飞行器。

让我们假设一个一维的世界。我们假设一架飞机正在朝向远离雷达（或朝向雷达）飞行。在一维世界中，雷达的角度是恒定的，飞机的高度是恒定的，如下图所示。

![](images/ex2_oneD_radar.png)

$x_n$ 表示在时间n时飞行航程的变化。飞机速度定义为距离相对于时间的变化率，因此速度是距离的导数：
$$\dot x = v = \frac{dx}{dt}$$
雷达以恒定速率向目标的方向发送跟踪光束。跟踪周期为 $\triangle t$

假设速度恒定，系统的动态模型可以用两个运动方程来描述：
$$x_{n+1}=x_n+\triangle t\dot x_n$$
$$\dot x_{n+1}=\dot x_n$$
根据方程可知，下一个追踪周期的飞机航程等于当前追踪周期的航程加上飞机速度乘以雷达追踪周期时间。在这个例子中，我们假设飞行速度是恒定的。因此，下一个周期的速度等于当前周期的速度。

上述方程组称为**状态外推方程State Extrapolation Equation**（也称为**过渡方程Transition Equation**或**预测方程Prediction Equation**），这是五个卡尔曼滤波方程之一。这个方程组将当前状态外推到下一个状态（预测）。

我们已经在前面的例子中使用了状态外推方程，我们假设下一个状态的重量等于当前状态的重量。

矩阵表示法中有一种状态外推方程的一般形式，我们稍后将学习。

在这个例子中，我们将使用上面的一组特定于我们的案例的方程。

Note: we have already learned two of the five Kalman filter equations:

- State Update Equation
- State Extrapolation Equation

设置雷达跟踪周期间隔 $\triangle t$ 为5ms。假设在n−1时，无人机（无人飞行器）的预计航程为30000m，速度为40m/s。

使用状态外推方程，我们可以预测时间n时目标的位置：
$$\hat x_{n,n-1}=\hat x_{n-1,n-1}+\triangle t(\hat{\dot x}_{n-1,n-1})=30000+5*40=30200m$$
时间n时的目标速度：
$$\hat{\dot x}_{n,n-1}=\hat{\dot x}_{n-1,n-1}=40m/s$$
然而，在时间n，雷达测量的范围（ $z_n$ ）为30110米，而不是预期的30200米。预期（预测）范围和测量范围之间有90米的间隙。造成这种差距的可能原因有两个：

- 雷达测量不精确
- 飞机速度改变了。新速度是 (30110-30000)/5=22m/s

写下速度的状态方程：
$$\hat{\dot x}_{n,n}=\hat{\dot x}_{n,n-1}+\beta (\frac{z_n-\hat{\dot x}_{n,n-1}}{\triangle t})$$
系数β的值取决于雷达的精确程度。假设雷达的1σ精度为20米，很可能是由于飞机速度的变化导致了预测距离和测量距离之间90米的差距。在这种情况下，我们将β因子设置为高值。如果我们将β设为0.9，则估计速度得到23.8m/s。

假设雷达的1σ精度为150m，很可能是雷达测量误差造成的50米差距。在这种情况下，我们将低值设置为β因子。如果我们将β设为0.1，则估计速度为38.2m/s。

飞机位置的状态更新方程与前一个例子中推导的方程类似，与以前的例子相反之处是，上一个例子中每次迭代需要重新计算α因子，在这个例子中α因子是常数。

α因子的大小取决于雷达测量精度。对于高精度雷达，我们将选择高α，这将给测量带来很高的权重。如果α=1，则估计范围等于测量范围；如果α=0，则测量没有意义。

因此，我们得到了一个方程组，它构成了雷达跟踪器的更新状态方程。它们也被称为**α-β轨迹更新方程(α−β track update equations)或α-β轨迹滤波方程(α−βtrack filtering equations)**。

The Update State Equation for position:
$$\hat x_{n,n}= \hat x_{n,n-1} + \alpha(z_n-\hat x_{n,n-1})$$

The Update State Equation for velocity:
$$\hat{\dot x}_{n,n}=\hat{\dot x}_{n,n-1}+\beta (\frac{z_n-\hat{x}_{n,n-1}}{\triangle t})$$
注：在一些书籍中，α−β过滤器称为g-h过滤器，其中希腊字母α替换为英文字母g，希腊字母β替换为英文字母h。
注：在本例中，我们从距离测量中求导得出飞机速度。现代雷达可以利用多普勒效应直接测量径向速度。然而，我的目标是解释卡尔曼滤波器而不是雷达操作。因此，为了一般性，在我们的例子中，我将继续从距离测量中推导速度。

考虑飞机在一维世界中正径向远离雷达（或朝向雷达）飞行。

α−β filter的参数是：
$$\alpha=0.2$$
$$\beta=0.1$$
雷达追踪周期每5秒更新一次。

注意：在这个例子中，我们将使用非常不精确的雷达和低速无人机（UAV）来更好地显示图形。在现实生活中，雷达通常更精确，目标也更快。

#### 第0次迭代

初始化：

当时间n=0时，初始条件如下：
$$\hat{x}_{0,0}=30000m$$
$$\hat{\dot x}_{0,0}=40m/s$$
注：追踪初始化Track Initiation（或我们如何获得初始条件）是一个非常重要的主题，稍后将讨论。现在或者目标是了解基本的α-β过滤器操作，所以假设初始条件是由其他人给出的。

预测：

应使用状态外推方程将初始猜测外推到第一个周期（n=1）：
$$\hat{x}_{n+1,n}=\hat{x}_{n,n}+\triangle t\hat{\dot x}_{n,n} = 30000 + 5*40 = 30200m$$
$$\hat{\dot x}_{n+1,n}=\hat{\dot x}_{n,n} = 40m/s$$

#### 第1次迭代

在第一个周期（n=1），最初的猜测是上一次的估计值：
$$\hat{x}_{n,n-1}=\hat{x}_{1,0} = 30200m$$
$$\hat{\dot x}_{n,n-1}=\hat{\dot x}_{1,0} = 40m/s$$
雷达测量飞机的距离：
$$z_1=30110m$$
使用状态更新方程计算当前估计值：
$$\hat x_{1,1}= \hat x_{1,0} + \alpha(z_1-\hat x_{1,0}) = 30182m$$
$$\hat{\dot x}_{1,1}=\hat{\dot x}_{1,0}+\beta (\frac{z_1-\hat{x}_{1,0}}{\triangle t}) = 38.2 m/s$$

#### 第2次至第10次迭代

后面也是一样的，可以看到，我们的估计算法对测量有平滑的影响（即，逐渐测量值的变化缩小）并且它收敛到真实值。

![](images/ex2_lowAlphaBeta.png)

（注意，prediction指状态外推得到的对下一个时段的估计，本时段利用上时段prediction和状态更新方程更新的结果是对本时段的estimation）

使用高α和β：

下图描述了α=8和β=0.5的真实值、测量值和估计值。

![](images/ex2_highAlphaBeta.png)

这个滤波器的“平滑度”要低得多。“当前估计”与实测值非常接近，预测误差较大。

那么我们应该一直选择α和β的低值吗？

答案是否定的。α和β的值取决于测量精度。如果我们使用非常精确的设备，比如激光雷达，我们会更喜欢高α和β的测量。在这种情况下，滤波器会对目标的速度变化作出快速响应。另一方面，如果测量精度较低，我们会选择较低的α和β。在这种情况下，滤波器将降低测量中的不确定度（误差）。然而，滤波器对目标速度变化的反应会慢得多。

由于α和β的计算是一个重要的课题，我们将在后面更详细地讨论它。

在这个例子中，我们推导了α-β滤波器状态更新方程。我们还学习了状态外推方程。本文提出了一种基于α-β滤波器的一维动态系统估计算法，并求解了匀速目标的数值案例。

### 案例3：在一维空间中追踪加速飞行器

在这个例子中，我们将用上一个例子中已经解释过的α−β滤波器跟踪以恒定加速度运动的飞机。

在前一个例子中，我们跟踪以40米/秒的恒定速度运动的无人机。下图描述了无人机航程和速度与时间的关系。

这个飞行器在前15秒中以50m/s的恒定速度运动，然后这个飞行器在后面的35秒中以8m/s的加速度运动。

将使用在上一个案例中已经使用过的α-β filter来追踪这个飞行器。

假设飞机在一维世界中正向雷达（或远离雷达）径向运动。

α-β filter的参数分别是：
$$\alpha=0.2$$
$$\beta=0.1$$
雷达追踪周期为5秒。

初始化

当时间n=0的初始条件是：
$$\hat{x}_{0,0}=30000m$$
$$\hat{\dot x}_{0,0}=50m/s$$
利用上面的状态更新和外推方程可以得到结果：

![](images/ex3_RangeVsTime.png)

![](images/ex3_VelocityVsTime.png)

可以看到真实值或测量值与估计值之间存在差值。这种差值称为滞后误差lag error。滞后误差的其他常见名称有：

- 动态误差 Dynamic error
- 系统误差 Systematic error
- 偏差误差 Bias error
- 截断误差 Truncation error

在这个案例中，我们检测到滞后误差是由加速度引起的。

### 案例4: 使用α−β−γ滤波器追踪加速度飞行器

α-β-γ滤波器（有时称为g-h-k滤波器）会将目标的加速度考虑进去，因此状态外推方程为：
$$\hat x_{n+1,n}=\hat x_{n,n}+\hat{\dot x}_{n,n}\triangle t+\hat{\ddot x}_{n,n}\frac{\triangle t^2}{2}$$
$$\hat{\dot x}_{n+1,n}=\hat{\dot x}_{n,n}+\hat{\ddot x}_{n,n}\triangle t$$
$$\hat{\ddot x}_{n+1,n}=\hat{\ddot x}_{n,n}$$
状态更新方程变成：
$$\hat x_{n,n}= \hat x_{n,n-1}+\alpha(z_n-\hat x_{n,n-1})$$
$$\hat{\dot x}_{n,n}= \hat{\dot x}_{n,n-1}+\beta(\frac{z_n-\hat x_{n,n-1}}{\triangle t})$$
$$\hat{\ddot x}_{n,n}= \hat{\ddot x}_{n,n-1}+\gamma(\frac{z_n-\hat x_{n,n-1}}{0.5\triangle t^2})$$
例子这里就不赘述了，总之是先用上一阶段对本时段的预测值，结合本时段观测，利用状态更新方程更新为本时段对本时段的估计，然后利用外推对下一个时段做预测，直接放结果。

![](images/ex4_RangeVsTime.png)

![](images/ex4_VelocityVsTime.png)

![](images/ex4_AccelerationVsTime.png)

如图所见，带有包含加速度的动态模型方程的α-β-γ滤波器可以以恒定加速度跟踪目标并降低滞后误差。

但是在目标是高度变化的情况下会发生什么呢？目标可以通过转弯突然改变飞行方向。真正的目标动态模型也可以包括一个突然加速或转弯（改变加速度）。在这种情况下，具有恒定α-β-γ系数的α-β-γ滤波器会产生估计误差，在某些情况下会追踪失效。

卡尔曼滤波器可以处理动态模型中的不确定性，这在后面再介绍。

有许多类型的 α-β-γ滤波器，它们基于相同的原理：

- 当前状态估计基于状态更新方程。
- 下一个状态估计（预测）是基于动态模型方程。

这些滤波器之间的主要区别在于加权系数α−β−（γ）的选择。有些过滤器类型使用恒定的加权系数，有些则为每个过滤器迭代（循环）计算加权系数。

α、β和γ的选择对于评估算法的正确功能至关重要。另一个重要问题是过滤器的初始化，即过滤器第一次迭代提供初始值。

以下列表包括最常用的α−β−（γ）过滤器：

- Wiener Filter
- Bayes Filter
- Fading-memory polynomial Filter
- Expanding-memory (or growing-memory) polynomial Filter
- Least-squares Filter
- Benedict–Bordner Filter
- Lumped Filter
- Discounted least-squares α−β Filter
- Critically damped α−β Filter
- Growing-memory Filter
- Kalman Filter
- Extended Kalman Filter
- Unscented Kalman Filter
- Extended Complex Kalman Filter
- Gauss-Hermite Kalman Filter
- Cubature Kalman Filter
- Particle Filter

## 一维卡尔曼滤波器

这个章节中描述在一维中使用卡尔曼滤波器。这个章节的主要目标是在不使用让人看起来困惑的数学工作的情况下，以一种更简单明了直接的方式解释卡尔曼滤波器。

我们将逐步学习卡尔曼滤波器的内容

- 首先，我们将为一个简单的例子推导卡尔曼滤波方程，没有过程噪声。
- 其次，我们将添加过程噪声。

正如我前面提到的，卡尔曼滤波器基于五个方程。我们已经熟悉其中两个：

- 状态更新方程。
- 动态模型方程（即状态外推方程）。

在这章，我们将推导其它三个卡尔曼滤波方程。

让我们回顾一下我们的第一个例子（金条重量测量），我们进行了多次测量，并通过求平均值来计算估计值。

我们得到如下结果：

![](images/ex1_MeasVsTrueVsEst.png)

在上面的图表中，您可以看到真实值、估计值和测量值，以及测量值的数量。

测量值（蓝色样本）和真实值（绿线）之间的差异是测量误差。由于测量误差是随机的，我们可以用方差（σ2）来描述。测量误差的方差可以由高度计供应商（altimeter vendor）提供，也可以通过校准程序得出。**测量误差的方差**实际上就是**测量不确定度measurement uncertainty**。

注：在一些文献中，测量不确定度（measurement uncertainty）也称为测量误差（measurement error）。

我们将使用r表示测量不确定度。

估计值（红线）与真实值（绿线）直接的差值就是**估计误差estimate error**。正如你看到的，随着测量次数越来越多，估计误差变得越来越小，当估计值向真实值收敛时，估计误差会收敛到0。我们不知道估计误差是多少，但是我们可以估算估计中的不确定性。

我们使用p表示估计不确定度。

让我们看看重量测量PDF（概率密度函数）【PDF (Probability Density Function)】。

在下面图中，我们将会看到金条重量的10次测量。

- 测量用蓝线描述
- 真实值用红间断线描述
- 绿线描述测量的概率密度函数
- 加粗的绿色区域是测量的标准偏差（σ），也就是说，真实值在该区域内的概率为68.26%。

如你所见，10个测量值中有8个与真实值足够接近，因此真值位于1σ边界内。

测量不确定度（r）是测量的方差（σ2）。

![](images/PDFs.png)

### 一维中的Kalman Gain Equation

我们将要推导的第三个方程就是Kalman Gain Equation。现在，我将展示Kalman Gain Equation的直觉的推导。数学推导在后续章节中展示。

在卡尔曼滤波器中，α-β(-γ)参数是在每次迭代过程中动态计算的。这些参数被成为卡尔曼收益（Kalman Gain），并且被记为： $K_n$。

卡尔曼收益（Kalman Gain）的方程如下：
$$K_n=\frac{Uncertainty\ in\ Estimate}{Uncertainty\ in\ Estimate + Uncertainty\ in\ Measurement}=\frac{p_{n,n-1}}{p_{n,n-1}+r_n}$$
其中，

$p_{n,n-1}$是extrapolated estimate uncertainty

$r_n$是measurement uncertainty

卡尔曼增益介于0到1之间。
$$0\leq K_n\leq 1$$
重写状态更新方程：
$$\hat x_{n,n}= \hat x_{n,n-1}+K_n(z_n-\hat x_{n,n-1})=(1-K_n)\hat x_{n,n-1}+K_nz_n$$
如你所见，卡尔曼增益Kalman Gain ($K_n$)是我们给到观测值（测量值measurement uncertainty）的权重，（1- $K_n$ )是我们给到估计值的权重。

当测量不确定度（measurement uncertainty）非常大且估计不确定度（estimate uncertainty）非常小时，卡尔曼收益Kalman Gain趋近于0.因此，我们估计值一个较大的权重，给测量值一个较小的权重。

另一方面，当测量不确定度（measurement uncertainty）非常小且估计不确定度（estimate uncertainty）非常大时，卡尔曼收益Kalman Gain趋近于1.因此我们给估计不确定度一个较小的权重，给测量不确定度一个较大的权重。

如果测量不确定度=估计不确定度，那么卡尔曼收益Kalman Gain=0.5.

卡尔曼增益告诉你，我想通过给定的测量值改变我的估计。

### 一维中估计不确定度更新estimate uncertainty update

下面公式定义了估计不确定度更新：

$$p_{n,n}=(1-K_n)p_{n,n-1}$$
其中，

$K_n$是Kalman Gain

$p_{n,n-1}$是the estimate uncertainty that was calculated during the previous filter estimation

$p_{n,n}$是the estimate uncertainty of the current sate

这个方程更新了当前状态的估计不确定度。这个方程也被叫做**协方差更新方程Covariance Update Equation**。为什么叫做协方差呢？在后续的章节中我们会详细了解。

由于（$1-K_n\leq 1$），我们可以很清楚的从方程中看到，随着滤波器的每次迭代，估计不确定性会变得越来越小。当测量不确定性增大，卡尔曼收益Kalman Gain会变小，因此，估计不确定度的收敛会变慢。然而，当测量不确定度变小，卡尔曼收益Kalman Gain会变大，估计不确定性会快速收敛到0.

### 一维中的估计不确定性外推estimate uncertainty extrapolation

与状态外推法一样，估计不确定性外推法也是用动态模型方程进行的。

以上一章节第二个例子为例，和动态模型方程类似地，估计不确定性外推将是：

$$p^x_{n+1,n}=p^x_{n,n}+\triangle t^2\cdot p^v_{n,n}$$
$$p^v_{n+1,n}=p^v_{n,n}$$
其中:

$p^x$	是 the position estimate uncertainty

$p^v$	是 the velocity estimate uncertainty

即预测位置估计不确定度等于当前位置估计不确定度加上当前速度估计不确定度乘以时间。预测的速度估计不确定性等于当前的速度估计不确定性（假设为恒速模型）。

估计不确定度外推方程称为**协方差外推方程Covariance Extrapolation Equation**，这是第五个卡尔曼滤波方程。

### 将所有方程放在一起 Putting all together

在一个算法中组合所有部分。与α，β，（γ）滤波器一样，卡尔曼滤波器采用了“测量，更新，预测”算法。下图提供了算法的简单示意图描述：

![](images/KalmanFilterAlgorithm.png)

滤波器的输入是：

- 初始化

初始化操作只有一次，它提供两个参数：

1) 初始系统状态($\hat x_1$,0)
2) 初始状态不确定度 ($p_1$,0)

初始化参数可以由另一个系统提供，另一个过程（例如，雷达中的搜索过程）或基于经验或理论知识的有根据的猜测。即使初始化参数不精确，卡尔曼滤波器也能够收敛到接近实际值。

- 测量

滤波器每次迭代都要执行一次测量，它提供两个参数：

1) 测量的系统状态（$z_n$）
2) 测量不确定度（$r_n$）

除了测量的值，卡尔曼滤波器需要测量不确定度参数。通常，这个参数由设备供应商提供，或者也可以测量设备标定中获取。雷达测量不确定度取决于多个参数，如SNR（信噪比）、光束宽带、带宽、观察目标时间、时钟稳定性等等。每个雷达测量器由不同的SNR（信噪比）、光束宽带、带宽、观察目标时间、时钟稳定性等。因此，该雷达计算每次测量的测量不确定度，并将其报告给跟踪器。

滤波器的输出是：

1) 系统状态估计 [$\hat x_{n,n}$]
2) 估计不确定度 [$p_{n,n}$]

除了系统状态估计，卡尔曼滤波器也提供了估计不确定度。

我们有估计精度的优点。正如我已经提到的，估算不确定度由以下公式给出：
$$p_{n,n}=(1-K_n)p_{n,n-1}$$

由于 $1-K_n\leq 1$ ,滤波器的每次迭代都会让 $p_{n,n}$ 越来越小。

所以这取决于我们要进行多少次测量。如果我们测量建筑高度，并且我们对3厘米（σ）的精度感兴趣，我们将进行测量，直到估计不确定度（σ2）小于9厘米。

|Equation	|Equation Name	|Alternative names used in the literature|
|-|-|-|
|$\hat x_{n,n}= \hat x_{n,n-1}+K_n(z_n-\hat x_{n,n-1})$	|State Update	|Filtering Equation|
|$\hat x_{n+1,n}=\hat x_{n,n}+\triangle t\hat{\dot x}_{n,n} \\ \hat{\dot x}_{n+1,n}=\hat{\dot x}_{n,n}$ (For constant velocity dynamics)|State Extrapolation	|Predictor Equation; Transition Equation; Prediction Equation; Dynamic Model; State Space Model|
|$Kn=\frac{p_{n,n}}{p_{n,n-1}+r_n}$	|Kalman Gain	|Weight Equation|
|$p_{n,n}= (1-K_n)p_{n,n-1}$	|Covariance Update	|Corrector Equation|
|$p_{n+1,n}=p_{n,n}$ (For constant dynamics)	|Covariance Extrapolation	|Predictor Covariance Equation|

注1：状态外推方程和协方差外推方程取决于系统动力学（每个系统的运动方程方式）。

注2：上表显示了针对特定情况定制的卡尔曼滤波方程的特殊形式。方程的一般形式将在后面的矩阵表示法中给出。现在，我们的目标是理解卡尔曼滤波器的概念。

下图提供了卡尔曼滤波器框图的详细说明。

![](images/DetailedKalmanFilterAlgorithm.png)

- step 0：初始化

与之前提到的一样，初始化操作只有一次，它提供两个参数：

1) 初始系统状态 ($\hat x_1$, 0)
2) 初始状态不确定度 ($p_1$, 0)

紧接着初始化步骤的是预测截断。

- step 1：测量

测量过程提供两个参数：

1) 测量的系统状态（$y_n$）
2) 测量不确定度（$r_n$）

- step 2: 状态更新

状态更新过程负责系统的当前状态估计。

状态更新阶段的输入是：

1) 已测量的值 （$y_{1,0}$）
2) 测量不确定度 （$r_n$）
3) 前一个系统状态估计 （$\hat x_{n,n-1}$）
4) 估计不确定度 （$p_{n,n-1}$）

基于这些输入，状态更新过程计算卡尔曼收益Kalman Gain并提供两个输出

1) 当前系统状态估计 （$\hat x_{n,n}$）
2) 上一个状态估计不确定度（$p_{n,n}$）

这些参数都是卡尔曼滤波器的输出。

- step 3 ：预测

预测过程基于系统的动态模型将当前系统状态和当前系统状态估计不确定度外推到下一个系统状态。

在第一次滤波器迭代时，初始化输出被视为先前状态估计和不确定性。

在下一次滤波器迭代中，预测输出变为先前状态估计和不确定性。

### 卡尔曼增益直觉性思考

卡尔曼增益定义了测量的权重和形成新估计时先前估计的权重。

- 高卡尔曼增益

与估计不确定度相关的一个低测量不确定度，将会产生一个高卡尔曼增益（趋近1）。结果是，新的估计非常接近测量值。下图展示了在飞行器追踪应用中高卡尔曼增益对估计的影响。

![](images/HighKalmanGain.png)

- 低卡尔曼增益

与估计不确定度相关的一个高的测量不确定度将会产生低卡尔曼增益（趋近于0）。结果是，新的估计与前一个估计非常相似。下图展示了在飞行器追踪应用中低卡尔曼增益对估计的影响。

![](images/LowKalmanGain.png)

### 案例5: 估计建筑高度

稍后补充

### 一维卡尔曼滤波器的完整模型

现在，我们将使用过程噪音变量更新协方差外推方程。

- 过程噪音（PROCESS NOISE）

现实世界中的系统动态模型具有很多的不确定性。例如，当我们想估计电阻器的阻力时，我们假设电阻器是一个恒定动态系统，即，阻力在两次测量之间没有发生变化。然而，如果环境温度发生变化，那么阻力也会有轻微的改变。当使用雷达追踪弹道导弹时，动态模型的不确定度包活目标加速度的随机变化。对于飞行器，由于可能的飞机尾迹，不确定性要大得多。

另一方面，如果我们使用GPS接收器估计估计一个静态物体的位置，由于静态物体不会移动，因此，这个静态物体的动态模型就是0。动态模型的不确定度被称为过程噪声Process Noise。在文献中，它还称为工厂噪声plant noise，驱动噪声riving noise，动态噪声dynamics noise，模型噪声model noise和系统噪声system noise。 过程噪声会产生估计误差。

**过程噪声方差Process Noise Variance**的字母表示：q

**协方差外推方程Covariance Extrapolation Equation**将会包含过程噪声方差Process Noise Variance。

恒定动态模型（指的是静态模型）的协方差外推方程是：
$$p_{n,n-1}=p_{n-1.n-1}+q_n$$
